# LLMS-Generator Experiments

For experimentation with the LLMS-Generator Agent.

## Setup

To run this notebook, ensure you've authenticated to Google Cloud and installed the project's dependencies.

To setup the Google Cloud environment:

```bash
source scripts/setup-env.sh
```

Then to install the package dependencies into the virtual environment, use the `uv` tool:

1. From your agent's root directory, run `make install` to set up the virtual environment (`.venv`).
2. In this Jupyter notebook, select the kernel from the `.venv` folder to ensure all dependencies are available.

### Import Libraries

In [ ]:
import os

import vertexai
from dotenv import load_dotenv
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.auth import default
from google.genai.types import Content, Part
from IPython.display import Markdown, display

### Load Environment Variables

In [ ]:
dotenv_path = os.path.abspath('../.env')

if os.path.exists(dotenv_path):
    print(f"Loading environment variables from: {dotenv_path}")
    load_dotenv(dotenv_path=dotenv_path)
else:
    print(f"Warning: .env file not found at {dotenv_path}")

staging_project_id = os.getenv("GOOGLE_CLOUD_STAGING_PROJECT")
if staging_project_id:
    os.environ["GOOGLE_CLOUD_PROJECT"] = staging_project_id
    print(f"Set GOOGLE_CLOUD_PROJECT environment variable to: {staging_project_id}")


### Ensure We're Authenticating to the Right Project

In [ ]:
credentials, project_id = default()  # To use ADC
vertexai.init(project=staging_project_id, location="europe-west4", credentials=credentials)

## Running the Agent

In [ ]:
from llms_gen_agent.agent import root_agent

In [ ]:
# Session and Runner
APP_NAME = "generate_llms_client"
USER_ID="test_user"
SESSION_ID="test_session"
TEST_REPO_PATH = "/home/darren/localdev/gcp/adk-docs" # You can change this to any repository you want to test

async def setup_session_and_runner():
    session_service = InMemorySessionService()
    session = await session_service.create_session(app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID)
    runner = Runner(agent=root_agent, app_name=APP_NAME, session_service=session_service)
    return session, runner

In [ ]:

# Agent Interaction
async def call_agent_async(query):
    content = Content(role='user', parts=[Part(text=query)])
    _, runner = await setup_session_and_runner()
    events = runner.run_async(user_id=USER_ID, session_id=SESSION_ID, new_message=content)

    async for event in events:
        if function_calls := event.get_function_calls():
            tool_name = function_calls[0].name
            display(Markdown(f"_Using tool {tool_name}..._"))
        elif event.actions and event.actions.transfer_to_agent:
            personality_name = event.actions.transfer_to_agent
            display(Markdown(f"_Delegating to agent: {personality_name}..._"))
        elif event.is_final_response() and event.content and event.content.parts:
            final_message = event.content.parts[0].text
            display(Markdown("### Final Message"))
            display(Markdown(final_message))


In [ ]:
query = f"Generate the llms.txt file for the repository at {TEST_REPO_PATH}"
await call_agent_async(query)